## Deep Learning Based Human Body Size Measurement Application

In [ ]:
# example inputs saved in the repo
image_route = 'front.jpg' 
side_image_route = 'side.jpg' 
output_route = 'output.png'
threshold=0.75 # default 0.75
body_height=170
#if the height is in inch multiply 2.54

fontsize = 0.6 #default 0.6
spacing = 25 #default 25

In [ ]:
import cv2
import tensorflow as tf
from tf_bodypix.api import load_model, download_model, BodyPixModelPaths
from tf_bodypix.bodypix_js_utils.part_channels import PART_CHANNELS

from scipy.spatial import distance as dist

import numpy as np
from matplotlib import pyplot as plt
import math
import time

start = time.time()
print("imported libraries")

bp_model = load_model(download_model(BodyPixModelPaths.RESNET50_FLOAT_STRIDE_16))

check = time.time() 
print(f"{check - start:.5f} sec")
print("loaded the model")

image = cv2.imread(image_route)

check_imageload = time.time()
print(f"{check_imageload - check:.5f} sec")
print("loaded the image")

prediction = bp_model.predict_single(image)
mask = prediction.get_mask(threshold=threshold)
part_segmentation = prediction.get_scaled_part_segmentation(mask, part_names=PART_CHANNELS.copy())

check1 = time.time() 
print(f"{check1 - check_imageload:.5f} sec")
print("the model processed the image")

body = np.where(part_segmentation!=-1)
face = np.where((part_segmentation==0)|(part_segmentation==1))
left_arm = np.where((part_segmentation==2)|(part_segmentation==3)|(part_segmentation==6)|(part_segmentation==7))
right_arm = np.where((part_segmentation==4)|(part_segmentation==5)|(part_segmentation==8)|(part_segmentation==9))
left_hand = np.where(part_segmentation==10)
right_hand = np.where(part_segmentation==11)
torso = np.where((part_segmentation==12)|(part_segmentation==13))
left_leg = np.where((part_segmentation==14)|(part_segmentation==15)|(part_segmentation==18)|(part_segmentation==19))
right_leg = np.where((part_segmentation==16)|(part_segmentation==17)|(part_segmentation==20)|(part_segmentation==21))
left_foot = np.where(part_segmentation==22)
right_foot = np.where(part_segmentation==23)
lower_body = np.where((part_segmentation==14)|(part_segmentation==15)|(part_segmentation==18)|(part_segmentation==19)|(part_segmentation==16)|(part_segmentation==17)|(part_segmentation==20)|(part_segmentation==21))

center = np.where((part_segmentation==12)|(part_segmentation==13)|(part_segmentation==14)|(part_segmentation==15)|(part_segmentation==18)|(part_segmentation==19)|(part_segmentation==16)|(part_segmentation==17)|(part_segmentation==20)|(part_segmentation==21))

if len(body[0]) > 0:
    pixel_height = np.amax(body[0]) - np.amin(body[0]) # bottom - top
    
else:
    raise NotImplementedError("BODY HASN'T BEEN DETECTED")
    
if len(face[0]) > 0:    
    pixel_facelength = np.amax(face[0]) - np.amin(face[0])
    pixel_facewidth = np.amax(face[1]) - np.amin(face[1])
else:
    pixel_facelength = None
    pixel_facewidth = None
    
if len(torso[0]) > 0:
    pixel_torsolength = np.amax(torso[0]) - np.amin(torso[0])
    pixel_shoulderwidth = np.amax(torso[1]) - np.amin(torso[1])
else:
    pixel_torsolength = None
    pixel_shoulderwidth = None
    
if len(left_leg[0]) > 0:
    pixel_leftleglength = np.amax(left_leg[0]) - np.amin(left_leg[0])
    pixel_leftlegwidth = np.amax(left_leg[1]) - np.amin(left_leg[1])
else:
    pixel_leftleglength = None
    pixel_leftlegwidth = None

if len(right_leg[0]) > 0:
    pixel_rightleglength = np.amax(right_leg[0]) - np.amin(right_leg[0])
    pixel_rightlegwidth = np.amax(right_leg[1]) - np.amin(right_leg[1])
else:
    pixel_rightleglength = None
    pixel_rightlegwidth = None

try: 
    if pixel_leftleglength >= pixel_rightleglength:
        pixel_leglength = pixel_leftleglength
    else:
        pixel_leglength = pixel_rightleglength
except:
    pixel_leglength = None

try:
    if pixel_leftlegwidth >= pixel_rightlegwidth:
        pixel_legwidth = pixel_leftlegwidth
    else:
        pixel_legwidth = pixel_rightlegwidth
except:
    pixel_legwidth = None

if (pixel_leftleglength is None) and (pixel_rightleglength is None):
    pixel_hipwidth = None
elif pixel_leftleglength is None:
    hippoint = np.amin(right_leg[0]) 
    pixel_hipwidth = np.amax(center[1][np.where(center[0] == hippoint)]) - np.amin(center[1][np.where(center[0] == hippoint)])
elif pixel_rightleglength is None:
    hippoint = np.amin(left_leg[0])
    pixel_hipwidth = np.amax(center[1][np.where(center[0] == hippoint)]) - np.amin(center[1][np.where(center[0] == hippoint)])    
else:    
    if np.amin(right_leg[0]) < np.amin(left_leg[0]):
        hippoint = np.amin(right_leg[0])
        pixel_hipwidth = np.amax(center[1][np.where(center[0] == hippoint)]) - np.amin(center[1][np.where(center[0] == hippoint)])
    else: 
        hippoint = np.amin(left_leg[0])
        pixel_hipwidth = np.amax(center[1][np.where(center[0] == hippoint)]) - np.amin(center[1][np.where(center[0] == hippoint)])    
while pixel_hipwidth < pixel_torsolength/4:  
    hippoint -= 1
    pixel_hipwidth = np.amax(center[1][np.where(center[0] == hippoint)]) - np.amin(center[1][np.where(center[0] == hippoint)])
            
height = body_height

colored = prediction.get_colored_part_mask(mask).astype(np.uint8)
masked_image = cv2.bitwise_and(image, colored)

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

parts = [body, torso, lower_body, left_leg, right_leg]

#drawing each part's boundary
for part in parts:
    if len(part[0]) != 0: 
        ul, ll, ur, lr = (np.amin(part[1]), np.amin(part[0])), (np.amin(part[1]), np.amax(part[0])), (np.amax(part[1]), np.amin(part[0])), (np.amax(part[1]), np.amax(part[0]))

        box = (ul, ur, ll, lr)
        cv2.rectangle(masked_image, ul, lr, (0, 0, 255), 2)

        (ulurX, ulurY) = midpoint(ul, ur)
        (lllrX, lllrY) = midpoint(ll, lr)

        (ulllX, ulllY) = midpoint(ul, ll)
        (urlrX, urlrY) = midpoint(ur, lr)

        cv2.circle(masked_image, (int(ulurX), int(ulurY)), 5, (255,0,0), -1)
        cv2.circle(masked_image, (int(lllrX), int(lllrY)), 5, (255,0,0), -1)
        cv2.circle(masked_image, (int(ulllX), int(ulllY)), 5, (255,0,0), -1)
        cv2.circle(masked_image, (int(ulurX), int(ulurY)), 5, (255,0,0), -1)
        cv2.line(masked_image, (int(ulurX), int(ulurY)), (int(lllrX), int(lllrY)), (255,0,255), 2)
    else:
        pass
    
#getting coordinates of arms (arms' length is differently calculated since it's not perpendicular to the ground)
arm_boxes = []
if len(left_arm[0]) > 0: 
    cnt_leftArm = []
    for i in range(len(left_arm[0])):
        x, y = left_arm[0][i], left_arm[1][i]
        cord = (y,x)
        cnt_leftArm.append(cord)
    
    cnt_leftArm = np.asarray(cnt_leftArm)
    
    rect1 = cv2.minAreaRect(cnt_leftArm)
    box1 = cv2.boxPoints(rect1)
    box1 = np.int0(box1)
    arm_boxes.append(box1)

if len(right_arm[0]) > 0:
    cnt_rightArm = []
    for i in range(len(right_arm[0])):
        x, y = right_arm[0][i], right_arm[1][i]
        cord = (y,x)
        cnt_rightArm.append(cord)
    
    cnt_rightArm = np.asarray(cnt_rightArm)
    
    rect2 = cv2.minAreaRect(cnt_rightArm)
    box2 = cv2.boxPoints(rect2)
    box2 = np.int0(box2)
    arm_boxes.append(box2)
    
if not len(arm_boxes) == 0:
    
    arm_lengths = []
    for b in arm_boxes:
        (tl, tr, br, bl) = b
        (tltrX, tltrY) = midpoint(tl, tr)
        (blbrX, blbrY) = midpoint(bl, br)

        (tlblX, tlblY) = midpoint(tl, bl)
        (trbrX, trbrY) = midpoint(tr, br)

        length1 = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
        length2 = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
        
        if length1 >= length2:
            length = length1
            ###
            cv2.circle(masked_image, (int(tltrX), int(tltrY)), 5, (255,0,0), -1)
            cv2.circle(masked_image, (int(blbrX), int(blbrY)), 5, (255,0,0), -1)
            cv2.line(masked_image, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)), (255,0,255), 2)
            ###
        else:
            length = length2
            ###
            cv2.circle(masked_image, (int(tlblX), int(tlblY)), 5, (255,0,0), -1)
            cv2.circle(masked_image, (int(trbrX), int(trbrY)), 5, (255,0,0), -1)
            cv2.line(masked_image, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)), (255,0,255), 2)
            ###            
            
        arm_lengths.append(length)
        cv2.drawContours(masked_image, [b], 0, (0,255,0), 2)
    pixel_armlength = max(arm_lengths)
else:
    pixel_armlength = None
    
gap = 0
pixelsPerMetric = pixel_height / height

cv2.putText(masked_image, "height: {:.1f}cm".format(height), (10,20+gap*spacing), 
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)

if pixel_shoulderwidth is not None:
    gap += 1
    shoulderwidth = pixel_shoulderwidth / pixelsPerMetric
    cv2.putText(masked_image, "shoulder width: {:.1f}cm".format(shoulderwidth), (10,20+gap*spacing), 
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
else:
    gap += 1
    cv2.putText(masked_image, "shoulder width: not detected", (10,20+gap*spacing),  
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
    
if pixel_leglength is not None:
    gap += 1
    leglength = pixel_leglength / pixelsPerMetric
    cv2.putText(masked_image, "leg length: {:.1f}cm".format(leglength), (10,20+gap*spacing), 
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
else:
    gap += 1
    cv2.putText(masked_image, "leg length: not detected", (10,20+gap*spacing),  
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)

if pixel_hipwidth is not None:
    gap += 1
    hipwidth = pixel_hipwidth / pixelsPerMetric
    cv2.putText(masked_image, "hip width: {:.1f}cm".format(hipwidth), (10,20+gap*spacing), 
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
else:
    gap += 1
    cv2.putText(masked_image, "hip width: not detected", (10,20+gap*spacing),  
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)

if pixel_armlength is not None:
    gap += 1
    armlength = pixel_armlength / pixelsPerMetric
    cv2.putText(masked_image, "arm length: {:.1f}cm".format(armlength), (10,20+gap*spacing), 
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
else:
    gap += 1
    cv2.putText(masked_image, "arm length: not detected", (10,20+gap*spacing),  
           cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)

check2 = time.time() 
print(f"{check2 - check1:.5f} sec")    
print("processed the first image...")    
    
if side_image_route is not None:
    side_image = cv2.imread(side_image_route)
    
    side_image = cv2.resize(side_image, (int(side_image.shape[1] * image.shape[0] / side_image.shape[0]), image.shape[0]),
                           interpolation=cv2.INTER_CUBIC)

    side_prediction = bp_model.predict_single(side_image)
    side_mask = side_prediction.get_mask(threshold=threshold)
    side_part_segmentation = side_prediction.get_scaled_part_segmentation(side_mask, part_names=PART_CHANNELS.copy())

    side_body = np.where(side_part_segmentation!=-1)
    side_face = np.where((side_part_segmentation==0)|(side_part_segmentation==1))
    side_left_arm = np.where((side_part_segmentation==2)|(side_part_segmentation==3)|(side_part_segmentation==6)|(side_part_segmentation==7))
    side_right_arm = np.where((side_part_segmentation==4)|(side_part_segmentation==5)|(side_part_segmentation==8)|(side_part_segmentation==9))
    side_left_hand = np.where(side_part_segmentation==10)
    side_right_hand = np.where(side_part_segmentation==11)
    side_torso = np.where((side_part_segmentation==12)|(side_part_segmentation==13))
    side_left_leg = np.where((side_part_segmentation==14)|(side_part_segmentation==15)|(side_part_segmentation==18)|(side_part_segmentation==19))
    side_right_leg = np.where((side_part_segmentation==16)|(side_part_segmentation==17)|(side_part_segmentation==20)|(side_part_segmentation==21))
    side_left_foot = np.where(side_part_segmentation==22)
    side_right_foot = np.where(side_part_segmentation==23)
    side_lower_body = np.where((side_part_segmentation==16)|(side_part_segmentation==17)|(side_part_segmentation==20)|(side_part_segmentation==21)|(side_part_segmentation==14)|(side_part_segmentation==18)|(side_part_segmentation==15)|(side_part_segmentation==19))
    
    if len(side_body[0]) > 0:
        side_pixel_height = np.amax(side_body[0]) - np.amin(side_body[0]) # bottom - top    
    else:
        raise NotImplementedError("BODY HASN'T BEEN DETECTED")
    
    if len(side_face[0]) > 0:    
        side_pixel_facelength = np.amax(side_face[0]) - np.amin(side_face[0])
        side_pixel_facewidth = np.amax(side_face[1]) - np.amin(side_face[1])
    else:
        side_pixel_facelength = None
        side_pixel_facewidth = None
    
    if len(side_torso[0]) > 0:
        side_pixel_torsolength = np.amax(side_torso[0]) - np.amin(side_torso[0])
        side_pixel_shoulderwidth = np.amax(side_torso[1]) - np.amin(side_torso[1])
    else:
        side_pixel_torsolength = None
        side_pixel_shoulderwidth = None
    
    if len(side_left_leg[0]) > 0:
        side_pixel_leftleglength = np.amax(side_left_leg[0]) - np.amin(side_left_leg[0])
        side_pixel_leftlegwidth = np.amax(side_left_leg[1]) - np.amin(side_left_leg[1])
    else:
        side_pixel_leftleglength = None
        side_pixel_leftlegwidth = None


    if len(side_right_leg[0]) > 0:
        side_pixel_rightleglength = np.amax(side_right_leg[0]) - np.amin(side_right_leg[0])
        side_pixel_rightlegwidth = np.amax(side_right_leg[1]) - np.amin(side_right_leg[1])
    else:
        side_pixel_rightleglength = None
        side_pixel_rightlegwidth = None

    try: 
        if side_pixel_leftleglength >= side_pixel_rightleglength:
            side_pixel_leglength = side_pixel_leftleglength
        else:
            side_pixel_leglength = side_pixel_rightleglength
    except:
        side_pixel_leglength = None

    try:
        if side_pixel_leftlegwidth >= side_pixel_rightlegwidth:
            side_pixel_legwidth = side_pixel_leftlegwidth
        else:
            side_pixel_legwidth = side_pixel_rightlegwidth
    except:
        side_pixel_legwidth = None

    try:    
        if (side_pixel_leftleglength is None) and (side_pixel_rightleglength is None):
            side_pixel_hipwidth = None
        elif side_pixel_leftleglength is None:
            side_hippoint = np.amin(side_right_leg[0])
            side_pixel_hipwidth = np.amax(side_body[1][np.where(side_body[0] == side_hippoint)]) - np.amin(side_body[1][np.where(side_body[0] == side_hippoint)])
        elif side_pixel_rightleglength is None:
            side_hippoint = np.amin(side_left_leg[0])
            side_pixel_hipwidth = np.amax(side_body[1][np.where(side_body[0] == side_hippoint)]) - np.amin(side_body[1][np.where(side_body[0] == side_hippoint)])    
        else:    
            if np.amin(side_right_leg[0]) < np.amin(side_left_leg[0]):
                side_hippoint = np.amin(side_right_leg[0])
                side_pixel_hipwidth = np.amax(side_body[1][np.where(side_body[0] == side_hippoint)]) - np.amin(side_body[1][np.where(side_body[0] == side_hippoint)])
            else: 
                side_hippoint = np.amin(side_left_leg[0])
                side_pixel_hipwidth = np.amax(side_body[1][np.where(side_body[0] == side_hippoint)]) - np.amin(side_body[1][np.where(side_body[0] == side_hippoint)])    
        while side_pixel_hipwidth < side_pixel_torsolength/4:
            side_hippoint -= 1
            side_pixel_hipwidth = np.amax(side_body[1][np.where(side_body[0] == side_hippoint)]) - np.amin(side_body[1][np.where(side_body[0] == side_hippoint)])
    except: 
        side_pixel_hipwidth = None

    side_height = body_height

    side_colored = side_prediction.get_colored_part_mask(side_mask).astype(np.uint8)
    side_masked_image = cv2.bitwise_and(side_image, side_colored)

#drawing each part's boundary
    parts = [side_body, side_torso, side_lower_body, side_left_leg, side_right_leg]
    for part in parts:
        if len(part[0]) != 0: 
            ul, ll, ur, lr = (np.amin(part[1]), np.amin(part[0])), (np.amin(part[1]), np.amax(part[0])), (np.amax(part[1]), np.amin(part[0])), (np.amax(part[1]), np.amax(part[0]))

            box = (ul, ur, ll, lr)
            cv2.rectangle(side_masked_image, ul, lr, (0, 0, 255), 2)

            (ulurX, ulurY) = midpoint(ul, ur)
            (lllrX, lllrY) = midpoint(ll, lr)

            (ulllX, ulllY) = midpoint(ul, ll)
            (urlrX, urlrY) = midpoint(ur, lr)

            cv2.circle(side_masked_image, (int(ulurX), int(ulurY)), 5, (255,0,0), -1)
            cv2.circle(side_masked_image, (int(lllrX), int(lllrY)), 5, (255,0,0), -1)
            cv2.circle(side_masked_image, (int(ulllX), int(ulllY)), 5, (255,0,0), -1)
            cv2.circle(side_masked_image, (int(ulurX), int(ulurY)), 5, (255,0,0), -1)

            cv2.line(side_masked_image, (int(ulurX), int(ulurY)), (int(lllrX), int(lllrY)), (255,0,255), 2)
        else:
            pass
    
#getting coordinates of arms (arms' length is differently calculated since it's not perpendicular to the ground)
    side_arm_boxes = []
    if len(side_left_arm[0]) > 0: 
        side_cnt_leftArm = []
        for i in range(len(side_left_arm[0])):
            x, y = side_left_arm[0][i], side_left_arm[1][i]
            cord = (y,x)
            side_cnt_leftArm.append(cord)

        side_cnt_leftArm = np.asarray(side_cnt_leftArm)

        rect1 = cv2.minAreaRect(side_cnt_leftArm)
        box1 = cv2.boxPoints(rect1)
        box1 = np.int0(box1)
        side_arm_boxes.append(box1)

    if len(side_right_arm[0]) > 0:
        side_cnt_rightArm = []
        for i in range(len(side_right_arm[0])):
            x, y = side_right_arm[0][i], side_right_arm[1][i]
            cord = (y,x)
            side_cnt_rightArm.append(cord)

        side_cnt_rightArm = np.asarray(side_cnt_rightArm)

        rect2 = cv2.minAreaRect(side_cnt_rightArm)
        box2 = cv2.boxPoints(rect2)
        box2 = np.int0(box2)
        side_arm_boxes.append(box2)
    
    if not len(arm_boxes) == 0:
        side_arm_lengths = []
        for b in side_arm_boxes:
            (tl, tr, br, bl) = b
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)

            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)

            length1 = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            length2 = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

            if length1 >= length2:
                length = length1
                ###
                cv2.circle(side_masked_image, (int(tltrX), int(tltrY)), 5, (255,0,0), -1)
                cv2.circle(side_masked_image, (int(blbrX), int(blbrY)), 5, (255,0,0), -1)
                cv2.line(side_masked_image, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)), (255,0,255), 2)
                ###
            else:
                length = length2
                ###
                cv2.circle(side_masked_image, (int(tlblX), int(tlblY)), 5, (255,0,0), -1)
                cv2.circle(side_masked_image, (int(trbrX), int(trbrY)), 5, (255,0,0), -1)
                cv2.line(side_masked_image, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)), (255,0,255), 2)
                ###            

            side_arm_lengths.append(length)  
            cv2.drawContours(side_masked_image, [b], 0, (0,255,0), 2)
        side_pixel_armlength = max(side_arm_lengths)  
    else:
        side_pixel_armlength = None
        
    gap = 0
    side_pixelsPerMetric = side_pixel_height / side_height
    cv2.putText(side_masked_image, "height: {:.1f}cm".format(side_height), (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)

    if side_pixel_shoulderwidth is not None:
        gap += 1
        side_shoulderwidth = side_pixel_shoulderwidth / side_pixelsPerMetric
        cv2.putText(side_masked_image, "shoulder depth: {:.1f}cm".format(side_shoulderwidth), (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
    else: 
        side_shoulderwidth = None
        gap += 1
        cv2.putText(side_masked_image, "shoulder depth: not detected", (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)          

    if side_pixel_leglength is not None:
        gap += 1
        side_leglength = side_pixel_leglength / side_pixelsPerMetric
        cv2.putText(side_masked_image, "leg length: {:.1f}cm".format(side_leglength), (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
    else:
        side_leglength = None
        gap += 1
        cv2.putText(side_masked_image, "leg length: not detected", (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)  

    if side_pixel_legwidth is not None:
        gap += 1
        side_legwidth = side_pixel_legwidth / side_pixelsPerMetric
        cv2.putText(side_masked_image, "leg width: {:.1f}cm".format(side_legwidth), (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
    else: 
        side_legwidth = None
        gap += 1
        cv2.putText(side_masked_image, "leg depth: not detected", (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)  

    if side_pixel_hipwidth is not None:
        gap += 1
        side_hipwidth = side_pixel_hipwidth / side_pixelsPerMetric
        cv2.putText(side_masked_image, "hip depth: {:.1f}cm".format(side_hipwidth), (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
    else:
        side_hipwidth = None
        gap += 1
        cv2.putText(side_masked_image, "hip depth: not detected", (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)          
        
    if side_pixel_armlength is not None:
        gap += 1
        side_armlength = side_pixel_armlength / side_pixelsPerMetric
        cv2.putText(side_masked_image, "arm length: {:.1f}cm".format(side_armlength), (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
    else:
        side_armlength = None
        gap += 1
        cv2.putText(side_masked_image, "arm length: not detected", (10,20+gap*spacing), 
               cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)      
    
    if pixel_torsolength is not None:
        torsolength = pixel_torsolength / pixelsPerMetric
    if side_pixel_torsolength is not None:
        side_torsolength = side_pixel_torsolength / side_pixelsPerMetric
    
    if (side_torsolength is not None) and (torsolength is not None):
        upper_side_torso = (side_torso[0][np.where(side_torso[0] < (np.amin(side_torso[0]) + side_pixel_torsolength/2))],
                            side_torso[1][np.where(side_torso[0] < (np.amin(side_torso[0]) + side_pixel_torsolength/2))])# upper half or torso
        lower_side_torso = (side_torso[0][np.where(side_torso[0] >= (np.amin(side_torso[0]) + side_pixel_torsolength/2))],
                            side_torso[1][np.where(side_torso[0] >= (np.amin(side_torso[0]) + side_pixel_torsolength/2))])# lower half of torso

        width = 0
        for row_num in np.unique(upper_side_torso[0]): #행마다 폭을 찾고
            wid = np.amax(upper_side_torso[1][np.where(upper_side_torso[0] == row_num)]) - np.amin(upper_side_torso[1][np.where(upper_side_torso[0] == row_num)])
            if wid >= width: 
                width = wid 
                row_chest = row_num      
        side_pixel_chestwidth = width
        new_row_chest = int((row_chest - np.amin(side_body[0])) * pixel_height / side_pixel_height + np.amin(body[0]))
        pixel_chestwidth = np.amax(torso[1][np.where(torso[0] == new_row_chest)]) - np.amin(torso[1][np.where(torso[0] == new_row_chest)])
                
        side_buttock_half = np.amax(lower_side_torso[0]) - np.amin(side_lower_body[0])
        row_waist = int(np.amin(side_lower_body[0]) - side_buttock_half)
        side_pixel_waistwidth = np.amax(side_body[1][np.where(side_body[0] == row_waist)]) - np.amin(side_body[1][np.where(side_body[0] == row_waist)])
        
        buttock_half = np.amax(torso[0]) - np.amin(lower_body[0])
        new_row_waist = int(np.amin(lower_body[0]) - buttock_half)
        pixel_waistwidth = np.amax(torso[1][np.where(torso[0] == new_row_waist)]) - np.amin(torso[1][np.where(torso[0] == new_row_waist)])

        cv2.line(side_masked_image, 
            (np.amax(side_torso[1][np.where(side_torso[0] == row_chest)]),row_chest),
            (np.amin(side_torso[1][np.where(side_torso[0] == row_chest)]),row_chest),
            (0,255,255),2)
        cv2.line(masked_image, 
            (np.amax(torso[1][np.where(torso[0] == new_row_chest)]),new_row_chest),
            (np.amin(torso[1][np.where(torso[0] == new_row_chest)]),new_row_chest),
            (0,255,255),2)                     
        cv2.line(side_masked_image, 
            (np.amax(side_body[1][np.where(side_body[0] == row_waist)]),row_waist),
            (np.amin(side_body[1][np.where(side_body[0] == row_waist)]),row_waist),
            (0,155,255),2)
        
        cv2.line(masked_image, 
            (np.amax(torso[1][np.where(torso[0] == new_row_waist)]),new_row_waist),
            (np.amin(torso[1][np.where(torso[0] == new_row_waist)]),new_row_waist),
            (0,155,255),2)        
                        
        chestwidth = pixel_chestwidth / pixelsPerMetric
        waistwidth = pixel_waistwidth / pixelsPerMetric
        
        side_chestwidth = side_pixel_chestwidth / side_pixelsPerMetric
        side_waistwidth = side_pixel_waistwidth / side_pixelsPerMetric
        
        chest_circum = 2 * math.pi * math.sqrt(((chestwidth/2)**2 + (side_chestwidth/2)**2)/2)
        chest_circum_inch = chest_circum / 2.54
        waist_circum = 2 * math.pi * math.sqrt(((waistwidth/2)**2 + (side_waistwidth/2)**2)/2)
        waist_circum_inch = waist_circum / 2.54
        
        gap += 1
        cv2.putText(side_masked_image, "-------------------", (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX,
                   fontsize, (255,255,255), 2)
        #chest circumference
        gap += 1
        cv2.putText(side_masked_image, "chest girth: {:.1f}cm({:.1f}in)".format(chest_circum, chest_circum_inch),
                   (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
        #waist circumference
        gap += 1
        cv2.putText(side_masked_image, "waist girth: {:.1f}cm({:.1f}in)".format(waist_circum, waist_circum_inch),
                   (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
    else: 
        #chest circumference
        gap += 1
        cv2.putText(side_masked_image, "chest girth: not detected".format(chest_circum, chest_circum_inch),
                   (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
        #waist circumference
        gap += 1
        cv2.putText(side_masked_image, "waist girth: not detected".format(waist_circum, waist_circum_inch),
                   (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)        
        
    #hip circumference    
    if side_hipwidth is not None:    
        hip_circum = 2 * math.pi * math.sqrt(((hipwidth/2)**2 + (side_hipwidth/2)**2)/2)
        hip_circum_inch = hip_circum / 2.54
        gap += 1
        cv2.putText(side_masked_image, "hip girth: {:.1f}cm({:.1f}in)".format(hip_circum, hip_circum_inch),
                   (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
    else:
        gap += 1
        cv2.putText(side_masked_image, "hip girth: not detected",
                   (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)

    #sleeve length 
    if armlength or side_armlength is not None:
        if (side_armlength and armlength) is not None:
            gap += 1
            cv2.putText(side_masked_image, 'sleeve length: {:.1f}cm({:.1f}in)'.format(np.mean([side_armlength, armlength]), np.mean([side_armlength, armlength])/2.54), #그냥 사이드에서 얻은 길이를 넣어버렸는데 그럼 안될거 같음. 오히려 정면 길이가 더정확한거 같기도... 
                        (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
        elif side_armlength is None:
            gap += 1
            cv2.putText(side_masked_image, "sleeve length: {:.1f}cm({:.1f}in)".format(armlength, armlength/2.54),
                        (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
        else:
            gap += 1
            cv2.putText(side_masked_image, "sleeve length: {:.1f}cm({:.1f}in)".format(side_armlength, side_armlength/2.54),
                        (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)                                    
    else:
        gap += 1
        cv2.putText(side_masked_image, "sleeve length: not detected",
                    (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
        
    #inseam 
    if side_leglength or leglength is not None:
        if (side_leglength and leglength) is not None:
            gap += 1
            cv2.putText(side_masked_image, "inseam: {:.1f}cm({:.1f}in)".format(max([side_leglength, leglength]), max([side_leglength, leglength])/2.54),
                       (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
        elif side_leglength is None:
            gap += 1
            cv2.putText(side_masked_image, "inseam: {:.1f}cm({:.1f}in)".format(leglength, leglength/2.54),
                        (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
        else:
            gap += 1
            cv2.putText(side_masked_image, "inseam: {:.1f}cm({:.1f}in)".format(side_leglength, side_leglength/2.54),
                        (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)
    else:
        gap += 1
        cv2.putText(side_masked_image, "inseam: not detected",
                    (10, 20+gap*spacing), cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255,255,255), 2)

    if pixel_hipwidth is not None:        
        cv2.line(masked_image, #lower_body? or body?
                (np.amax(center[1][np.where(center[0] == hippoint)]),hippoint),
                (np.amin(center[1][np.where(center[0] == hippoint)]),hippoint),
                (0,255,255),2)              
                
    if side_pixel_hipwidth is not None:        
        cv2.line(side_masked_image, #lower_body 대신 선은 body and side_body
            (np.amax(side_body[1][np.where(side_body[0] == side_hippoint)]),side_hippoint),
            (np.amin(side_body[1][np.where(side_body[0] == side_hippoint)]),side_hippoint),
            (0,255,255),2)        

    check3 = time.time() 
    print(f"{check3 - check2:.5f} sec")    
    print("processed the second image...")           
        
    #size check- adding later#
    #concatenate horizontally
    masked_image = cv2.hconcat([masked_image, side_masked_image])

end = time.time()
print(f"{end - start:.5f} sec")
    
cv2.imwrite(output_route, masked_image)
cv2.namedWindow('demo', cv2.WND_PROP_FULLSCREEN)
cv2.imshow('image', image)
cv2.imshow('demo', masked_image)
cv2.waitKey(0)
cv2.destroyAllWindows()